In [2]:
import multiprocessing as mp
import random
from datetime import datetime

random.seed(12345)


### Computing $\pi$

In [3]:
nsamples = int(2e7)

def sample():
    x = random.uniform(-1.0, 1.0)
    y = random.uniform(-1.0, 1.0)
    if x**2 + y**2 <= 1:
        return 1
    else:
        return 0

#### Serial Computation

In [4]:
def sample_serial(nsamples):
    return sum(sample() for i in range(nsamples))

start = datetime.now()
hits = sample_serial(nsamples)
pi = 4.0 * hits / nsamples
print(f"Time: {datetime.now() - start}, pi: {pi:.9f}")

Time: 0:00:11.089793, pi: 3.141349200


#### Multithreading

In [6]:
from threading import Thread

def thread_sample(chunk_size, output, i):
    output[i] = sample_serial(chunk_size)

nsamples = int(2e7)
nthreads = 4
chunk_size = int(nsamples / nthreads)

output = [None] * nthreads
threads = []

start = datetime.now()
for i in range(nthreads):
    threads += [Thread(target = thread_sample, args=(chunk_size, output, i))]

for i in range(nthreads):
    threads[i].start()

for i in range(nthreads):
    threads[i].join()

hits = sum(output)
pi = 4.0 * hits / nsamples
print(f"Time: {datetime.now() - start}, pi: {pi:.9f}")

Time: 0:00:26.584747, pi: 3.141604600


#### Using Multiprocessing

##### Process class

In [32]:
import multiprocessing as mp

def process_sample(chunk_size, q):
    q.put(sample_serial(chunk_size))

nprocesses = 4
chunk_size = int(nsamples / nprocesses)

start = datetime.now()
q = mp.Queue()
processes = [mp.Process(target=process_sample, args=(chunk_size, q)) for x in range(nprocesses)]

for p in processes:
    p.start()

for p in processes:
    p.join()

hits = sum(q.get() for p in processes)
pi = 4.0 * hits / nsamples

print(f"Time: {datetime.now() - start}, pi: {pi:.9f}")

Time: 0:00:03.015545, pi: 3.141825800


##### Pool Class

In [34]:
# Blocks until function returns

nsamples = int(2e7)
nsplits = 20
nprocesses = 4
chunk_size = int(nsamples / nsplits)

def pool_sample(chunk_size):
    return sample_serial(chunk_size)

start = datetime.now()
pool = mp.Pool(processes = nprocesses)
results = [pool.apply(pool_sample, args=(chunk_size,)) for i in range(nsplits)]
hits = sum(results)
pi = 4.0 * hits / nsamples
print(f"Time: {datetime.now() - start}, pi: {pi:.9f}")

Time: 0:00:12.035188, pi: 3.141758800


In [33]:
# perform many function calls asynchronously

nsamples = int(2e7)
nsplits = 20
nprocesses = 4
chunk_size = int(nsamples / nsplits)

def pool_sample(chunk_size):
    return sample_serial(chunk_size)

start = datetime.now()
pool = mp.Pool(processes = nprocesses)
results = [pool.apply_async(pool_sample, args=(chunk_size,)) for i in range(nsplits)]
hits = sum(p.get() for p in results)
pi = 4.0 * hits / nsamples
print(f"Time: {datetime.now() - start}, pi: {pi:.9f}")

Time: 0:00:03.073790, pi: 3.142109400


In [42]:
# Map

args = [chunk_size for i in range(nsplits)]

start = datetime.now()
pool = mp.Pool(processes = nprocesses)
results = pool.map_async(pool_sample, args)
hits = sum(results.get())
pi = 4.0 * hits / nsamples
print(f"Time: {datetime.now() - start}, pi: {pi:.9f}")


Time: 0:00:03.710573, pi: 3.141283600
